<a href="https://colab.research.google.com/github/Lmalviya/machineTranslationTask/blob/main/chatbot_using_gpt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scope of this notebook
1. Fine tuen GPT2 model for Mental Health Counseling Chat bot

In [1]:
!pip install datasets
!pip install -U transformers

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel

import warnings
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
# import torchmetrics

In [3]:
def get_config():
  return {
      'datasetName': "Amod/mental_health_counseling_conversations",
      'split': 'train',
      'max_len': 1024,
      'batch_size': 2,
      'shuffle':True,
      'lr': 1e-04,
      'num_epochs': 2,
  }

In [4]:
class DataPreprocess(Dataset):
  def __init__(self, data, tokenizer, max_len:int = 1024):
    super().__init__()

    self.data = data
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    items = self.data[idx]
    context = items['Context']
    response =  items['Response']

    input_text = f"{context}\n [ANSWER]: {response}"

    input_tokens = self.tokenizer(input_text, return_tensors='pt')

    pad_size = self.max_len - input_tokens['input_ids'].size(1)


    input_ids = torch.cat((
        input_tokens['input_ids'],
        torch.tensor([self.tokenizer.encode('[PAD]')]*pad_size, dtype=torch.int64).view(1, pad_size)
    ), -1)


    input_mask = torch.cat((
        input_tokens['attention_mask'],
        torch.tensor([0]*pad_size, dtype=torch.int64).view(1, pad_size)
    ), -1)

    return {
        'input_ids': input_ids,
        'input_mask': input_mask,
        'context': context,
        'responce': response
    }

In [5]:
def get_data(config, tokenizer):
  raw_data = load_dataset(config['datasetName'], split=config['split'])
  num_of_data_points = len(raw_data)

  train_size = int(0.8*num_of_data_points)
  val_size = num_of_data_points - train_size
  train_raw, val_raw = random_split(raw_data, [train_size, val_size])

  train_class = DataPreprocess(train_raw, tokenizer, config['max_len'])
  val_class = DataPreprocess(val_raw, tokenizer, config['max_len'])

  train_ds = DataLoader(train_class, batch_size=config['batch_size'], shuffle=config['shuffle'])
  val_ds = DataLoader(val_class, batch_size=config['batch_size'], shuffle=config['shuffle'])

  return train_ds, val_ds



In [6]:
def get_model_and_tokenizer():
  tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
  tokenizer.add_special_tokens({
      'pad_token': '[PAD]',
      'bos_token': '[SOS]',
      'eos_token': '[EOS]'
  })
  tokenizer.add_tokens('[ANSWER]:')

  model = GPT2LMHeadModel.from_pretrained('gpt2')
  model.resize_token_embeddings(len(tokenizer))
  return model, tokenizer

In [7]:
def evaluate(model, data, device):
  model.eval()

  batch_iterator = tqdm(data)

  cumulative_loss = 0.0
  for batch in batch_iterator:
    input_ids = data['input_ids'].to(device)
    input_mask = data['input_mask'].to(device)
    loss = model(input_ids, attention_mask=input_mask, labels=input_ids).loss
    batch_iterator.set_postfix({'val loss': f"{loss.item():6.3f}"})
    cumulative_loss += loss.item()

  return cumulative_loss/len(data)


In [8]:
def train_model(config):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  print(f"Current device is used: {device}")

  model, tokenizer = get_model_and_tokenizer()
  model.to(device)

  train_ds, val_ds = get_data(config, tokenizer)

  # Tensorboard
  writer = SummaryWriter('Mental Health Counseling Chat Boat')
  optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)

  global_step = 0

  for epoch in range(config['num_epochs']):
    torch.cuda.empty_cache()
    model.train()

    batch_iterator = tqdm(train_ds, desc=f"Processing Epoch: {epoch:02d}")
    for batch in batch_iterator:
      optimizer.zero_grad()
      input_ids = batch['input_ids'].to(device) # (batch, seq_len)
      input_mask = batch['input_mask'].to(device) #(batch, seq_len)

      loss = model(input_ids, attention_mask=input_mask, labels=input_ids).loss
      batch_iterator.set_postfix({ "train loss": f"{loss.item():6.3f}" })

      writer.add_scalar("train loss", loss.item(), epoch)
      writer.flush()

      loss.backward()
      optimizer.step()

    val_loss = evaluate(model, val_ds, device)
    writer.add_scalar("val loss", loss.item(), epoch)
    writer.flush()

    return writer


In [9]:
config = get_config()
wt = train_model(config)

Current device is used: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Processing Epoch: 00:   0%|          | 0/1405 [00:00<?, ?it/s]

551
631


Processing Epoch: 00:   0%|          | 1/1405 [00:01<43:48,  1.87s/it, train loss=58.012]

934
634


...

Processing Epoch: 00:   3%|▎         | 45/1405 [00:31<15:44,  1.44it/s, train loss=1.489]

771
806


Processing Epoch: 00:   3%|▎         | 46/1405 [00:32<15:56,  1.42it/s, train loss=1.110]


987
-147


RuntimeError: invalid shape dimension -147

In [ ]:
# torch.save({
#         'epoch': epoch,
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict': optimizer.state_dict(),
#         'global_step': global_step
#     }, model_file_path)